# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [106]:
# Card and Deck Classes

import random 

class Card:
    suits = ["Clubs", "Hearts", "Spades", "Diamonds"]
    values = ["Ace",2,3,4,5,6,7,8,9,10,"J","Q","K"]

    def __init__(self, value, suit) :
        self.value = value
        self.suit = suit

    def red_joker():
        return Card("Joker", "Red")

    def black_joker():
        return Card("Joker", "Black")

    def is_ace(self):
        return self.value == "Ace"

    def is_joker(self):
        return self.value == "Joker"

    def is_face_card(self):
        return self.value == "J" or self.value == "Q" or self.value == "K"

    def is_numbered_card(self):
        try:
            int(self.value)
        except:
            return False
        else:
            return True

    def __str__(self):
        if self.is_joker():
            return f"{self.suit} {self.value}"
        else:
            return f"{self.value} of {self.suit}"

    def __repr__(self):
        return f"Card({self.__str__()})"

class Deck:

    def __init__(self, has_jokers = False):
        self.open_cards = []
        self.closed_cards = []
        self.has_jokers = has_jokers
        self.reset_deck()

    def reset_deck(self):
        self.open_cards = []
        self.closed_cards = []
        for suit in Card.suits:
            for value in Card.values:
                self.closed_cards.append(Card(value, suit))
        if self.has_jokers:
            self.closed_cards.append(Card.black_joker())
            self.closed_cards.append(Card.red_joker())

    def shuffle(self):
        random.shuffle(self.closed_cards)

    def draw(self):
        if len(self.closed_cards) == 0:
            raise Exception("Out of Cards exception")
        else:
            card = self.closed_cards.pop()
            self.open_cards.append(card)
            return card

    def __str__(self):
        return f"Open cards: {self.open_cards}, Closed cards: {self.closed_cards}"

    def __repr__(self):
        return f"Deck({self.__str__()})" 

In [63]:
# Hand Class

class Hand:

    def __init__(self):
        self.cards_in_hand = []
    
    def add_card(self, card):
        self.cards_in_hand.append(card)

    def blackjack_value(self):
        total = 0
        for card in self.cards_in_hand:
            if card.is_numbered_card():
                total += card.value
            elif card.is_ace():
                total += 11
            else:
                total += 10

        if total > 21:
            for card in self.cards_in_hand:
                if card.is_ace():
                    total -= 10
                    if total <= 21:
                        break
        return total

    def __repr__(self):
        return f"Hand({self.cards_in_hand})" 

    def show_first_only(self):
        if len(self.cards_in_hand) >= 2:
            rep = f"Hand([{self.cards_in_hand[0].__repr__()}"
            for i in range(len(self.cards_in_hand) - 1):
                rep+= ", Card(x of x)"
            rep += "])"
            return rep
        else: 
            return self.__repr__()

In [64]:
# Player Class

class Player:
    def __init__(self, name, chips):
        self.name = name
        self.chips = chips
        self.hand = Hand()

    def add_chips(self, amount):
        self.chips += amount

    def remove_chips(self, amount):
        if amount > self.chips:
            raise Exception("Not enought chips!")
        self.chips -= amount

    def has_enought_chips(self, amount):
        return amount <= self.chips

    def new_hand(self):
        self.hand = Hand()

In [173]:
# Game Class

class Game:
    def __init__(self, player):
        self.player = player
        self.deck = Deck()
        self.bet = 0
        self.dealer_hand = Hand()
        self.player.hand = Hand()
        self.game_on = True

    def place_bet(self, amount):
        if self.game_on and self.player.has_enought_chips(amount):
            self.player.remove_chips(amount)
            self.bet += amount

    def reset_game(self):
        self.deck.shuffle()
        self.dealer_hand = Hand()
        self.player.hand = Hand()
        self.dealer_hand.add_card(self.deck.draw())
        self.dealer_hand.add_card(self.deck.draw())
        self.player.hand.add_card(self.deck.draw())
        self.player.hand.add_card(self.deck.draw())
        self.game_on = True

    def player_hit(self):
        if self.game_on:
            self.player.hand.add_card(self.deck.draw())

        if self.did_player_bust():
            self.game_on = False

    def player_stand(self):
        self.game_on = False
        if not self.did_player_bust():
            self.dealer_plays()

    def dealer_plays(self):
        while self.dealer_hand.blackjack_value() < self.player.hand.blackjack_value() and not self.did_dealer_bust():
            self.dealer_hand.add_card(self.deck.draw())

    def did_player_bust(self):
        return self.player.hand.blackjack_value() > 21

    def did_dealer_bust(self):
        return self.dealer_hand.blackjack_value() > 21

    def did_player_win(self):
        return not self.did_player_bust() and (self.did_dealer_bust() or self.player.hand.blackjack_value() > self.dealer_hand.blackjack_value())

    def did_dealer_win(self):
        return not self.did_player_win()

    def finish_game(self):
        if self.did_player_win():
            self.player.add_chips(self.bet*2)
    
    def print_game(self):
        print(f'''
        ---------------------------------------------------------------------------
        Dealer : (Total = x, {self.dealer_hand.show_first_only()})
        

            Bet: {self.bet} chips
        {self.player.name} : (Total = {self.player.hand.blackjack_value()}, {self.player.hand})
        ---------------------------------------------------------------------------
        ''')
    
    def print_end_game(self):
        print(f'''
        ---------------------------------------------------------------------------
        Dealer : (Total = {self.dealer_hand.blackjack_value()}, {self.dealer_hand})
        

            Bet: {self.bet} chips
        {self.player.name} : (Total = {self.player.hand.blackjack_value()}, {self.player.hand})
        ---------------------------------------------------------------------------
        ''')




In [187]:
# The game

def ask_for_bet():
    while True:
        try:
            amount = int(input("How much you want to bet?"))
            return amount
        except:
            pass
        else:
            print("Sorry that is not a valid amount")

def ask_hit_stand():
    while True:
        action = input("Wanna (H)it or (S)tand?").upper()
        if action == "H" or action == "S":
            return action
        else:
            print("Sorry that is not a valid action")

def replay():
    i = input("Wanna play again, Y/N?").upper()
    return i == "Y"

def lets_play():

    name = input("What is your name?")
    p = Player(name, 1000)
    game = Game(p)

    while True:
        game.reset_game()

        while True:
            amount = ask_for_bet()
            if p.has_enought_chips(amount):
                game.place_bet(amount) 
                break
            else:
                print(f"Sorry, you only have {p.chips} chips left.")

                

        while game.game_on:
            game.print_game()
            action = ask_hit_stand()
            if action == "H":
                print("Player Hit!")
                game.player_hit()
            else:
                print("Player Stand!")
                game.player_stand()

        game.finish_game()
        game.print_end_game()
        if game.did_player_win():
            print("Congrats you won!") 
        elif game.did_player_bust():
            print("Sorry you bust =/")
        else:
            print("Sorry you lose =/")

        if not replay():
            break

    

In [188]:
lets_play()


        ---------------------------------------------------------------------------
        Dealer : (Total = x, Hand([Card(K of Hearts), Card(x of x)]))
        

            Bet: 50 chips
        lol : (Total = 17, Hand([Card(10 of Clubs), Card(7 of Hearts)]))
        ---------------------------------------------------------------------------
        
Player Hit!

        ---------------------------------------------------------------------------
        Dealer : (Total = 20, Hand([Card(K of Hearts), Card(Q of Diamonds)]))
        

            Bet: 50 chips
        lol : (Total = 26, Hand([Card(10 of Clubs), Card(7 of Hearts), Card(9 of Clubs)]))
        ---------------------------------------------------------------------------
        
Sorry you bust =/
